In [1]:
import os
os.chdir("../")
%pwd

'd:\\EndToEnd\\MLFLOW\\mlflow-end-to-end-0713'

In [2]:
from dataclasses import dataclass
from pathlib import Path

In [3]:
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str


In [18]:
from mlproject_0713.utils.common import read_yaml, create_directories
from mlproject_0713.constants import *

class ConfigurationManager:
    def __init__(
            self,
            config_path = CONFIG_FILE_PATH,
            schema_path = SCHEMA_FILE_PATH,
            params_path = PARAMS_FILE_PATH
            ):
          self.config = read_yaml(config_path)
          self.params = read_yaml(params_path)
          self.schema = read_yaml(schema_path)
          create_directories([Path(self.config.artifacts_root)])
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
         config = self.config.model_trainer
         params = self.params.ElasticNet
         schema = self.schema.TARGET_COLUMN
         create_directories([Path(config.root_dir)])
         model_trainer_config = ModelTrainerConfig(
              root_dir = config.root_dir,
              train_data_path = config.train_data_path,
              test_data_path = config.test_data_path,
              model_name = config.model_name,
              alpha = params.alpha,
              l1_ratio = params.l1_ratio,
              target_column = schema.name
         )
         return model_trainer_config

In [19]:
import pandas as pd
import os
from mlproject_0713 import logger
from sklearn.linear_model import ElasticNet
import joblib


In [25]:
class ModelTrainer:
    def __init__(self,config: ModelTrainerConfig):
        self.config = config
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop([self.config.target_column],axis=1)
        test_x = test_data.drop([self.config.target_column],axis=1)

        train_y = train_data[self.config.target_column]
        test_y = test_data[self.config.target_column]

        lr = ElasticNet(alpha = self.config.alpha, l1_ratio = self.config.l1_ratio, random_state = 42)
        lr.fit(train_x,train_y)

        joblib.dump(lr,os.path.join(self.config.root_dir,self.config.model_name))


In [26]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config = model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e


[2024-07-29 11:36:55,166: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-29 11:36:55,169: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-29 11:36:55,172: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-07-29 11:36:55,175: INFO: common: Created Directory at:artifacts]
[2024-07-29 11:36:55,176: INFO: common: Created Directory at:artifacts\model_trainer]
